## 01. 包装器Wrappers

`gymnasium.vector.VectorWrapper` 类提供了一个标准的接口来修改和扩展向量化环境的行为，而无需直接修改原始环境代码。通过继承 `VectorWrapper` 并重写特定的方法，用户可以轻松地添加新的功能，例如观察和奖励的转换、动作的修改、环境的重置逻辑等。以下是如何使用 `VectorWrapper` 类以及如何为向量化环境实现自定义包装器的示例。

### 示例代码

假设我们想要创建一个自定义的向量化环境包装器，该包装器会在每个时间步骤结束时将奖励乘以2。首先，我们需要定义我们的包装器类，重写 `step` 方法以修改奖励：

```python
import gymnasium as gym
from gymnasium.vector import VectorWrapper

class DoubleRewardVectorWrapper(VectorWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def step(self, actions):
        observations, rewards, terminations, truncations, infos = self.env.step(actions)
        # 将奖励乘以2
        rewards *= 2
        return observations, rewards, terminations, truncations, infos

    def reset(self, *, seed=None, options=None):
        return self.env.reset(seed=seed, options=options)

    def render(self, mode="human"):
        return self.env.render(mode=mode)

    def close(self):
        self.env.close()
```

接下来，我们创建一个向量化环境并将其包装在我们的 `DoubleRewardVectorWrapper` 中：

```python
# 创建向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 应用我们的自定义包装器
wrapped_envs = DoubleRewardVectorWrapper(envs)

# 重置环境并获取初始观察
observations, infos = wrapped_envs.reset(seed=42)
print(f"Initial observations: {observations}")

# 执行一步动作并观察奖励变化
actions = [1, 0, 1]  # 假设的动作
observations, rewards, terminations, truncations, infos = wrapped_envs.step(actions)
print(f"Rewards: {rewards}")

# 关闭环境
wrapped_envs.close()
```

在这个示例中，`DoubleRewardVectorWrapper` 包装器接受一个向量化环境作为输入，并通过重写 `step` 方法来修改奖励。我们创建一个 `CartPole-v1` 的向量化环境实例，将其传递给我们的包装器，然后通过调用 `reset` 和 `step` 方法来测试包装器的行为。

这种方式使得扩展和修改向量化环境变得非常灵活和方便，同时保持了代码的清晰和模块化。

`gymnasium.vector.VectorObservationWrapper` 类是为向量化环境设计的，允许开发者通过重写 `observations` 方法来自定义观察值的转换。这使得在不直接修改原始环境代码的情况下，可以轻松地对观察值进行修改或增强，类似于非向量化环境中的 `gymnasium.ObservationWrapper`。

### 示例代码

假设我们有一个向量化环境，我们想要将所有观察值标准化，使其均值为0且标准差为1。我们可以通过创建一个继承自 `VectorObservationWrapper` 的自定义包装器来实现这一点，然后重写 `observations` 方法来应用标准化转换：

```python
import numpy as np
from gymnasium.vector import VectorObservationWrapper

class NormalizeVectorObservation(VectorObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def observations(self, observations):
        # 计算观察值的均值和标准差
        mean = np.mean(observations, axis=0)
        std = np.std(observations, axis=0) + 1e-8  # 防止除以0
        # 应用标准化
        normalized_observations = (observations - mean) / std
        return normalized_observations
```

使用这个包装器的示例：

```python
import gymnasium as gym

# 创建一个向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 应用我们的标准化观察值包装器
normalized_envs = NormalizeVectorObservation(envs)

# 重置环境
observations, _ = normalized_envs.reset(seed=42)
print(f"Normalized initial observations: {observations}")

# 执行一些动作
actions = envs.action_space.sample()
observations, _, _, _, _ = normalized_envs.step(actions)
print(f"Normalized observations after step: {observations}")

# 关闭环境
normalized_envs.close()
```

在这个示例中，`NormalizeVectorObservation` 包装器计算每一步观察值的均值和标准差，然后应用标准化。这种方法可以轻松地应用于任何向量化环境，允许对观察值进行自定义的预处理，而无需更改原始环境代码。

`gymnasium.vector.VectorActionWrapper` 类是针对向量化环境设计的，使得开发者可以通过重写 `actions` 方法来自定义动作的转换。这为在不直接修改原始环境代码的情况下，轻松地对动作进行修改或增强提供了便利，类似于非向量化环境中的 `gymnasium.ActionWrapper`。

### 示例代码

假设我们有一个向量化环境，并希望对发送给环境的动作进行缩放处理。我们可以通过创建一个继承自 `VectorActionWrapper` 的自定义包装器来实现这一点，然后重写 `actions` 方法来应用缩放转换：

```python
import numpy as np
from gymnasium.vector import VectorActionWrapper

class ScaleVectorAction(VectorActionWrapper):
    def __init__(self, env, scale_factor=0.5):
        super().__init__(env)
        self.scale_factor = scale_factor
    
    def actions(self, actions):
        # 应用缩放
        scaled_actions = actions * self.scale_factor
        return scaled_actions
```

使用这个包装器的示例：

```python
import gymnasium as gym

# 创建一个向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 应用我们的动作缩放包装器，假设我们想要将所有动作值缩小一半
scaled_envs = ScaleVectorAction(envs, scale_factor=0.5)

# 重置环境
observations, _ = scaled_envs.reset(seed=42)

# 生成一些随机动作
actions = envs.action_space.sample()
print(f"Original actions: {actions}")

# 执行缩放后的动作
observations, _, _, _, _ = scaled_envs.step(actions)
print(f"Scaled actions applied")

# 关闭环境
scaled_envs.close()
```

在这个示例中，`ScaleVectorAction` 包装器将所有发送到环境的动作值缩小了一半。这种方法可以轻松地应用于任何向量化环境，允许对动作进行自定义的预处理，而无需更改原始环境代码。

`gymnasium.vector.VectorRewardWrapper` 类允许开发者对向量化环境中的奖励进行修改或转换，类似于非向量化环境中的 `gymnasium.RewardWrapper`。通过继承该类并重写 `rewards` 方法，可以轻松地对环境返回的奖励应用自定义的转换。

### 示例代码

假设我们有一个向量化环境，并希望将所有奖励值翻倍。我们可以创建一个继承自 `VectorRewardWrapper` 的自定义包装器类，然后重写 `rewards` 方法来实现奖励的翻倍：

```python
from gymnasium.vector import VectorRewardWrapper
import numpy as np

class DoubleVectorReward(VectorRewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def rewards(self, rewards):
        # 将奖励值翻倍
        doubled_rewards = rewards * 2
        return doubled_rewards
```

使用这个包装器的示例：

```python
import gymnasium as gym

# 创建一个向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 应用我们的奖励翻倍包装器
doubled_envs = DoubleVectorReward(envs)

# 重置环境
observations, _ = doubled_envs.reset(seed=42)

# 生成一些随机动作
actions = envs.action_space.sample()

# 执行动作并打印翻倍后的奖励
observations, rewards, _, _, _ = doubled_envs.step(actions)
print(f"Doubled rewards: {rewards}")

# 关闭环境
doubled_envs.close()
```

在这个示例中，`DoubleVectorReward` 包装器将所有从环境返回的奖励值翻倍。通过这种方式，可以轻松地对向量化环境的奖励进行自定义的预处理，而无需更改原始环境代码。

## 02.Vector Only wrappers

`gymnasium.wrappers.vector.DictInfoToList` 是一个向量化环境的包装器，它将环境返回的 `info` 格式从字典转换为字典列表。这对于那些预期 `info` 以列表格式返回的场景非常有用，例如，当您需要将环境的信息与某些旧代码或库集成时，而这些代码或库可能是基于列表的 `info` 格式编写的。

### 功能
此包装器主要用于将向量化环境中的 `info` 从单一字典（其中的每个键对应一个包含所有环境信息的 NumPy 数组）转换为一个字典列表（每个环境对应一个包含其信息的字典）。

### 参数
- `env (Env)`: 要应用此包装器的向量化环境。

### 示例代码

在下面的示例中，我们将创建一个向量化环境，并使用 `DictInfoToList` 包装器转换其 `info` 格式。然后，我们将执行一次环境的 `reset` 和 `step` 操作，并打印出转换后的 `info`：

```python
import gymnasium as gym
from gymnasium.wrappers.vector import DictInfoToList

# 创建一个向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3)

# 应用 DictInfoToList 包装器
envs = DictInfoToList(envs)

# 重置环境并获取初始观察和信息
observations, infos = envs.reset(seed=123)
print("Initial infos:", infos)

# 执行一步动作并获取新的信息
actions = envs.action_space.sample()
_, _, _, _, infos = envs.step(actions)
print("Infos after one step:", infos)
```

### 输出示例
```
Initial infos: [{}, {}, {}]
Infos after one step: [{}, {}, {}]
```

在这个示例中，由于我们没有在环境中执行任何会产生额外信息的操作，所以 `info` 列表中只包含空字典。但这展示了 `info` 格式如何从原始的向量化字典格式转换为每个子环境对应一个字典的列表格式。

### 注意事项
- 当使用其他对 `info` 进行操作的包装器（如 `RecordEpisodeStatistics`）时，`DictInfoToList` 需要作为最外层的包装器。
- 这个包装器的存在是为了兼容性和集成的目的，如果您的应用场景或库要求 `info` 以列表格式返回，这将非常有用。

`gymnasium.wrappers.vector.VectorizeTransformObservation` 是一个用于向量化环境的包装器，允许对单个代理环境中的观察进行转换的包装器应用于向量化环境。这使得用户可以对环境返回的观察进行自定义的转换操作，如修改观察的形状、数据类型或者应用某种函数变换等。

### 功能
此包装器通过将单代理的观察转换包装器应用于向量化环境中的每个子环境，来实现自定义的观察转换。它特别适用于需要对观察数据进行预处理或转换的情况。

### 参数
- `env`: 要包装的向量化环境。
- `wrapper`: 要向量化的观察转换包装器。
- `**kwargs`: 传递给观察转换包装器的关键字参数。

### 示例代码

下面的示例演示了如何使用 `VectorizeTransformObservation` 包装器来将环境返回的观察数据复制一份，即每个观察都变成了原始观察数据的两份拼接：

```python
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Box
from gymnasium.wrappers import TransformObservation
from gymnasium.wrappers.vector import VectorizeTransformObservation

# 创建向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 获取原始观察空间
old_space = envs.single_observation_space

# 定义新的观察空间，即将原始观察空间复制一份
new_space = Box(low=np.array([old_space.low, old_space.low]), high=np.array([old_space.high, old_space.high]))

# 应用 VectorizeTransformObservation 包装器，将观察数据复制一份
envs = VectorizeTransformObservation(envs, wrapper=TransformObservation, func=lambda x: np.array([x, x]), observation_space=new_space)

# 重置环境并获取观察
obs, info = envs.reset(seed=123)

# 关闭环境
envs.close()

# 打印观察结果
print(obs)
```

### 输出示例
```
array([[[ 0.01823519, -0.0446179 , -0.02796401, -0.03156282],
        [ 0.01823519, -0.0446179 , -0.02796401, -0.03156282]],

       [[ 0.02852531,  0.02858594,  0.0469136 ,  0.02480598],
        [ 0.02852531,  0.02858594,  0.0469136 ,  0.02480598]],

       [[ 0.03517495, -0.000635  , -0.01098382, -0.03203924],
        [ 0.03517495, -0.000635  , -0.01098382, -0.03203924]]],
      dtype=float32)
```

这个示例演示了如何将 `TransformObservation` 包装器应用于向量化环境，实现观察数据的自定义转换。通过 `VectorizeTransformObservation` 包装器，我们可以轻松地对向量化环境中的所有子环境执行相同的观察转换操作。

`gymnasium.wrappers.vector.VectorizeTransformAction` 是一个用于向量化环境的包装器，允许对单个代理环境中的动作转换包装器应用于向量化环境。这意味着用户可以自定义如何修改或转换向量化环境中的动作数据。

### 功能
此包装器通过将单代理的动作转换包装器应用于向量化环境中的每个子环境，来实现自定义的动作转换。它特别适用于需要对动作数据进行预处理或转换的情况。

### 参数
- `env`: 要包装的向量化环境。
- `wrapper`: 要向量化的动作转换包装器。
- `**kwargs`: 传递给动作转换包装器的关键字参数。

### 示例代码

下面的示例演示了如何使用 `VectorizeTransformAction` 包装器来将环境中的动作数据通过ReLU函数转换，即只保留动作数据中的正数部分，负数部分置为0：

```python
import gymnasium as gym
from gymnasium.wrappers import TransformAction
from gymnasium.wrappers.vector import VectorizeTransformAction

# 创建向量化环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 应用 VectorizeTransformAction 包装器，使用ReLU函数转换动作数据
envs = VectorizeTransformAction(envs, wrapper=TransformAction, func=lambda x: (x > 0.0) * x, action_space=envs.single_action_space)

# 重置环境并获取观察
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)

# 执行动作并获取新的观察
obs, rew, term, trunc, info = envs.step(envs.action_space.sample())

# 关闭环境
envs.close()

# 打印观察结果
print(obs)
```

### 输出示例
```
array([[-4.6343064e-01,  9.8971417e-05],
       [-4.4354835e-01, -5.9898634e-04],
       [-4.3034542e-01, -6.9532328e-04]], dtype=float32)
```

这个示例演示了如何将 `TransformAction` 包装器应用于向量化环境，实现动作数据的自定义转换。通过 `VectorizeTransformAction` 包装器，我们可以轻松地对向量化环境中的所有子环境执行相同的动作转换操作。

`gymnasium.wrappers.vector.VectorizeTransformReward` 是用于向量化环境的包装器，允许将单个代理环境中的奖励转换包装器应用于向量化环境。这意味着用户可以自定义如何修改或转换向量化环境中的奖励。

### 功能
此包装器通过将单代理的奖励转换包装器应用于向量化环境中的每个子环境，来实现自定义的奖励转换。它特别适用于需要对奖励数据进行预处理或转换的情况，例如奖励标准化、削减或任何自定义逻辑。

### 参数
- `env`: 要包装的向量化环境。
- `wrapper`: 要向量化的奖励转换包装器。
- `**kwargs`: 传递给奖励转换包装器的关键字参数。

### 示例代码

下面的示例演示了如何使用 `VectorizeTransformReward` 包装器来将环境中的奖励数据通过ReLU函数转换，即只保留奖励数据中的正数部分，负数部分置为0：

```python
import gymnasium as gym
from gymnasium.wrappers import TransformReward
from gymnasium.wrappers.vector import VectorizeTransformReward

# 创建向量化环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 应用 VectorizeTransformReward 包装器，使用ReLU函数转换奖励数据
envs = VectorizeTransformReward(envs, wrapper=TransformReward, func=lambda x: (x > 0.0) * x)

# 重置环境并获取观察
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)

# 执行动作并获取新的观察、奖励
obs, rew, term, trunc, info = envs.step(envs.action_space.sample())

# 关闭环境
envs.close()

# 打印奖励
print(rew)
```

### 输出示例
```
array([-0., -0., -0.])
```

这个示例演示了如何将 `TransformReward` 包装器应用于向量化环境，实现奖励数据的自定义转换。通过 `VectorizeTransformReward` 包装器，我们可以轻松地对向量化环境中的所有子环境执行相同的奖励转换操作。

## 03.Vectorized Common wrappers

`gymnasium.wrappers.vector.RecordEpisodeStatistics` 是用于向量化环境的包装器，用于追踪并记录每个子环境的累积奖励和剧集长度。它在每个剧集结束时，将剧集统计信息添加到 `info` 字典中，使用 `episode` 和 `_episode` 键来标识。

### 参数
- `env`: 要包装的向量化环境。
- `buffer_length`: 存储最近奖励和剧集长度的缓冲区大小，默认为100。
- `stats_key`: 用于保存数据的 `info` 键，默认为 `episode`。

### 示例代码

下面的代码演示了如何使用 `RecordEpisodeStatistics` 包装器来跟踪和记录向量化环境中所有子环境的累积奖励和剧集长度，并如何访问这些统计信息：

```python
from pprint import pprint
import gymnasium as gym

# 创建向量化环境并应用 RecordEpisodeStatistics 包装器
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")
envs = gym.wrappers.vector.RecordEpisodeStatistics(envs, buffer_length=10)

# 重置环境
obs, info = envs.reset(seed=123)
_ = envs.action_space.seed(123)

# 运行环境直到所有子环境结束或被截断
while True:
    obs, rew, term, trunc, info = envs.step(envs.action_space.sample())
    # 检查是否有任何子环境已结束或被截断
    if term.any() or trunc.any():
        break

# 关闭环境
envs.close()

# 打印最后一次迭代的信息，包含每个子环境的累积奖励和剧集长度
pprint(info)
```

### 输出示例

输出将显示在每个子环境结束或被截断时的累积奖励和剧集长度，例如：

```
{
    "_episode": array([True, False, False]),
    "episode": {
        "r": array([11.,  0.,  0.]),  # 累积奖励
        "l": array([11,  0,  0]),    # 剧集长度
        "t": array([0.007812, 0., 0.])  # 从剧集开始的经过时间
    },
    ...
}
```

这个示例演示了如何使用 `RecordEpisodeStatistics` 包装器来追踪向量化环境中所有子环境的累积奖励和剧集长度，并在每个子环境结束时将这些信息添加到 `info` 字典中。

## 04.Implemented Observation wrappers

`gymnasium.wrappers.vector.TransformObservation` 包装器用于通过提供的函数转换向量化环境中的观测值。这允许对观测值进行手动规范化处理，例如通过并行处理或其他优化方法处理向量观测值。当可以优化处理向量观测值时，使用此包装器；否则，应使用 `VectorizeTransformObservation`，在该包装器中只需要定义单个观测值的转换函数。

### 参数
- `env`: 要包装的向量化环境。
- `func`: 转换向量观测值的函数。如果这个转换的观测值超出了 `env.observation_space` 的范围，则需要提供一个新的 `observation_space`。
- `observation_space`: 包装器的观测空间。如果为 `None`，则假定与 `env.observation_space` 相同。

### 示例代码

下面的代码展示了如何在向量化环境中应用观测值转换函数来缩放和平移观测值，并指定新的观测空间：

```python
import gymnasium as gym
from gymnasium.spaces import Box

# 定义缩放和平移观测值的函数
def scale_and_shift(obs):
    return (obs - 1.0) * 2.0

# 创建向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 定义新的观测空间
new_obs_space = Box(low=envs.observation_space.low, high=envs.observation_space.high)

# 应用 TransformObservation 包装器
envs = gym.wrappers.vector.TransformObservation(envs, func=scale_and_shift, observation_space=new_obs_space)

# 重置环境并获取转换后的观测值
obs, info = envs.reset(seed=123)

# 打印转换后的观测值
print(obs)

# 关闭环境
envs.close()
```

### 输出示例
转换后的观测值可能类似于以下数组，这取决于具体的转换函数和初始观测值：
```
[[-1.9635296, -2.0892358, -2.055928 , -2.0631256],
 [-1.9429494, -1.9428282, -1.9061728, -1.9503881],
 [-1.9296501, -2.00127  , -2.0219676, -2.0640786]]
```

这个例子说明了如何使用 `TransformObservation` 包装器在向量化环境中对观测值进行自定义的转换处理，并指定新的观测空间。

`gymnasium.wrappers.vector.FilterObservation` 是一个向量化环境的包装器，它允许通过指定的键或索引来过滤字典或元组观测空间。这对于从环境中选择特定的观测值部分非常有用，尤其是当环境的观测值包含了多余的信息时。

### 参数
- `env`: 要包装的向量化环境。
- `filter_keys`: 要包含的子空间的键或索引。对于字典观测空间使用字符串列表，对于元组观测空间使用整数列表。

### 示例代码

以下代码演示了如何创建一个包含字典观测空间的向量化环境，并使用 `FilterObservation` 包装器来仅保留指定的键：

```python
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Dict, Box
from gymnasium.wrappers import TransformObservation
from gymnasium.wrappers.vector import VectorizeTransformObservation, FilterObservation

# 创建向量化环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 定义一个函数，将观测值转换为包含"obs"和"junk"的字典
make_dict = lambda x: {"obs": x, "junk": np.array([0.0])}
new_space = Dict({"obs": envs.single_observation_space, "junk": Box(low=-1.0, high=1.0)})

# 使用 VectorizeTransformObservation 包装器来应用转换函数
envs = VectorizeTransformObservation(env=envs, wrapper=TransformObservation, func=make_dict, observation_space=new_space)

# 使用 FilterObservation 包装器来过滤观测空间，仅保留"obs"键
envs = FilterObservation(envs, ["obs"])

# 重置环境并获取过滤后的观测值
obs, info = envs.reset(seed=123)

# 关闭环境
envs.close()

# 打印过滤后的观测值
print(obs)
```

### 输出示例

执行上述代码后，`obs` 将只包含 `FilterObservation` 包装器指定的键（在这个例子中是 `"obs"`）的观测值：

```
OrderedDict([('obs', array([[ 0.01823519, -0.0446179 , -0.02796401, -0.03156282],
       [ 0.02852531,  0.02858594,  0.0469136 ,  0.02480598],
       [ 0.03517495, -0.000635  , -0.01098382, -0.03203924]],
      dtype=float32))])
```

这个例子说明了如何在向量化环境中使用 `FilterObservation` 包装器过滤观测值，以便于聚焦于环境中的特定部分。

`gymnasium.wrappers.vector.FlattenObservation` 是一个观测包装器，它将向量化环境中的多维观测值平铺为一维数组。这对于将环境观测值直接输入到需要一维输入的模型中非常有用，如全连接神经网络。

### 参数
- `env`: 要包装的向量化环境。

### 示例代码

以下代码展示了如何使用 `FlattenObservation` 包装器将 `CarRacing-v2` 环境中的观测值平铺为一维数组：

```python
import gymnasium as gym

# 创建一个包含三个子环境的向量化 CarRacing-v2 环境
envs = gym.make_vec("CarRacing-v2", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值形状
obs, info = envs.reset(seed=123)
print(f"Original observation shape: {obs.shape}")

# 应用 FlattenObservation 包装器
envs = gym.wrappers.vector.FlattenObservation(envs)

# 重置环境并获取平铺后的观测值形状
obs, info = envs.reset(seed=123)
print(f"Flattened observation shape: {obs.shape}")

# 关闭环境
envs.close()
```

### 输出示例

执行上述代码后，您将看到观测值从原始形状 `(3, 96, 96, 3)` 被平铺为 `(3, 27648)`。这意味着每个子环境的观测值都被转换成了一个一维数组，其中 `3` 代表向量化环境中的子环境数量。

```
Original observation shape: (3, 96, 96, 3)
Flattened observation shape: (3, 27648)
```

这个例子说明了如何在向量化环境中使用 `FlattenObservation` 包装器来平铺观测值，从而使它们适用于需要一维输入数据的模型。

`gymnasium.wrappers.vector.GrayscaleObservation` 是一个观测包装器，用于将向量化环境中的 RGB 图像观测值转换为灰度图像。此包装器对于降低图像处理的复杂性和计算需求非常有用，特别是在图像色彩信息对于任务不是必需的情况下。

### 参数
- `env`: 要包装的向量化环境。
- `keep_dim` (可选): 指示在灰度图像中是否保持通道维度的布尔值。如果设置为 `True`，输出观测值的形状将包含一个单一的通道维度；如果为 `False`，则通道维度将被压缩，结果为二维图像。

### 示例代码

下面的示例代码展示了如何使用 `GrayscaleObservation` 包装器将 `CarRacing-v2` 环境中的 RGB 图像观测值转换为灰度图像，并展示了在保持和不保持通道维度情况下的形状变化。

```python
import gymnasium as gym
from gymnasium.wrappers.vector import GrayscaleObservation

# 创建一个包含三个子环境的向量化 CarRacing-v2 环境
envs = gym.make_vec("CarRacing-v2", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值形状
obs, info = envs.reset(seed=123)
print(f"Original observation shape: {obs.shape}")

# 应用 GrayscaleObservation 包装器，不保持通道维度
envs = GrayscaleObservation(envs, keep_dim=False)
obs, info = envs.reset(seed=123)
print(f"Grayscale observation shape (without keeping channel dim): {obs.shape}")

# 应用 GrayscaleObservation 包装器，保持通道维度
envs = GrayscaleObservation(envs, keep_dim=True)
obs, info = envs.reset(seed=123)
print(f"Grayscale observation shape (keeping channel dim): {obs.shape}")

# 关闭环境
envs.close()
```

### 输出示例

```
Original observation shape: (3, 96, 96, 3)
Grayscale observation shape (without keeping channel dim): (3, 96, 96)
Grayscale observation shape (keeping channel dim): (3, 96, 96, 1)
```

这个例子展示了如何在向量化环境中使用 `GrayscaleObservation` 包装器将 RGB 图像观测值转换为灰度图像，并根据 `keep_dim` 参数的值保留或移除通道维度。

`gymnasium.wrappers.vector.ResizeObservation` 是一个观测包装器，用于调整向量化环境中图像观测值的大小。这在你需要将观测值输入到具有特定输入尺寸要求的神经网络中时特别有用。

### 参数
- `env`: 要包装的向量化环境。
- `shape`: 一个包含两个整数的元组，指定调整后的图像观测值的新尺寸。

### 示例代码

以下示例展示了如何将 `CarRacing-v2` 环境中的图像观测值从原始尺寸 `(96, 96, 3)` 调整为 `(28, 28, 3)` 的新尺寸。

```python
import gymnasium as gym
from gymnasium.wrappers.vector import ResizeObservation

# 创建一个包含三个子环境的向量化 CarRacing-v2 环境
envs = gym.make_vec("CarRacing-v2", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值形状
obs, info = envs.reset(seed=123)
print(f"Original observation shape: {obs.shape}")

# 应用 ResizeObservation 包装器并调整观测值尺寸
envs = ResizeObservation(envs, shape=(28, 28))
obs, info = envs.reset(seed=123)
print(f"Resized observation shape: {obs.shape}")

# 关闭环境
envs.close()
```

### 输出示例

```
Original observation shape: (3, 96, 96, 3)
Resized observation shape: (3, 28, 28, 3)
```

这个例子展示了如何使用 `ResizeObservation` 包装器调整向量化环境中的图像观测值的尺寸。通过指定新的尺寸 `(28, 28)`，你可以将图像的大小调整为适合你的模型输入要求。

`gymnasium.wrappers.vector.ReshapeObservation` 是一个观测包装器，用于将向量化环境中的数组型观测值重塑为指定的形状。这在你需要将观测值输入到需要特定形状输入的模型时非常有用。

### 参数
- `env`: 要包装的向量化环境。
- `shape`: 重塑后的观测值形状。可以是一个整数或一个整数元组。

### 示例代码

以下示例展示了如何将 `CarRacing-v2` 环境中的图像观测值从原始形状 `(96, 96, 3)` 重塑为 `(9216, 3)` 的新形状。

```python
import gymnasium as gym
from gymnasium.wrappers.vector import ReshapeObservation

# 创建一个包含三个子环境的向量化 CarRacing-v2 环境
envs = gym.make_vec("CarRacing-v2", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值形状
obs, info = envs.reset(seed=123)
print(f"Original observation shape: {obs.shape}")

# 应用 ReshapeObservation 包装器并重塑观测值形状
envs = ReshapeObservation(envs, shape=(9216, 3))
obs, info = envs.reset(seed=123)
print(f"Reshaped observation shape: {obs.shape}")

# 关闭环境
envs.close()
```

### 输出示例

```
Original observation shape: (3, 96, 96, 3)
Reshaped observation shape: (3, 9216, 3)
```

这个例子展示了如何使用 `ReshapeObservation` 包装器重塑向量化环境中的图像观测值形状。通过指定新的形状 `(9216, 3)`，你可以将原始的 3D 图像数据重塑为一个二维数组，这可能有助于输入到某些类型的神经网络模型中。

此外，这个包装器不仅限于图像数据。任何类型的数组型观测值都可以被重塑，这让它成为一个在多种场景下都非常有用的工具。例如，如果你有一个环境输出的观测值是一个扁平的数组，但你的模型需要一个具有特定维度的数组（比如，在时间序列分析中），那么 ReshapeObservation 就可以帮你轻松地实现这个转换。

记得，使用这个包装器时，要确保你重塑后的观测值仍然在原环境的观测空间中。这意味着重塑后的观测值仍需符合环境定义的观测空间约束。这一点在设置 shape 参数时需要特别注意。

`gymnasium.wrappers.vector.RescaleObservation` 是一个观测包装器，用于将向量化环境中的观测值线性地缩放到指定的最小值和最大值之间。这在你需要将观测值标准化到特定范围内以提高学习算法的稳定性和效率时非常有用。

### 参数
- `env`: 要包装的向量化环境。
- `min_obs`: 观测值的新最小边界。可以是浮点数、整数或与观测空间形状相同的 NumPy 数组。
- `max_obs`: 观测值的新最大边界。可以是浮点数、整数或与观测空间形状相同的 NumPy 数组。

### 示例代码

以下示例展示了如何使用 `RescaleObservation` 包装器将 `CartPole-v1` 环境中的观测值从原始范围线性缩放到 `[-5.0, 5.0]` 的新范围内。

```python
import gymnasium as gym
from gymnasium.wrappers.vector import RescaleObservation

# 创建一个包含三个子环境的向量化 CartPole-v1 环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值范围
obs, info = envs.reset(seed=123)
print(f"Original min observation value: {obs.min()}")
print(f"Original max observation value: {obs.max()}")

# 应用 RescaleObservation 包装器并调整观测值范围
envs = RescaleObservation(envs, min_obs=-5.0, max_obs=5.0)
obs, info = envs.reset(seed=123)
print(f"Rescaled min observation value: {obs.min()}")
print(f"Rescaled max observation value: {obs.max()}")

# 关闭环境
envs.close()
```

### 输出示例

```
Original min observation value: -0.0446179
Original max observation value: 0.0469136
Rescaled min observation value: -0.33379582
Rescaled max observation value: 0.55998987
```

这个例子展示了如何使用 `RescaleObservation` 包装器调整向量化环境中的观测值范围。通过指定新的最小值和最大值，你可以将原始观测值线性缩放到新的范围内，这有助于提高模型的学习效率和稳定性。

`gymnasium.wrappers.vector.DtypeObservation` 是一个观测包装器，用于将向量化环境中的观测值数据类型转换为指定的 `dtype`。这在你需要将观测值的数据类型与模型或算法的要求相匹配时非常有用。

### 参数
- `env`: 要包装的向量化环境。
- `dtype`: 观测值的新数据类型。可以是任何有效的 NumPy `dtype`。

### 示例代码

以下示例展示了如何使用 `DtypeObservation` 包装器将 `CartPole-v1` 环境中的观测值数据类型从 `float32` 转换为 `float64`。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import DtypeObservation

# 创建一个包含三个子环境的向量化 CartPole-v1 环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 重置环境并获取原始的观测值数据类型
obs, info = envs.reset(seed=123)
print(f"Original observation dtype: {obs.dtype}")

# 应用 DtypeObservation 包装器并改变观测值的数据类型
envs = DtypeObservation(envs, dtype=np.float64)
obs, info = envs.reset(seed=123)
print(f"New observation dtype: {obs.dtype}")

# 关闭环境
envs.close()
```

### 输出示例

```
Original observation dtype: float32
New observation dtype: float64
```

这个例子展示了如何使用 `DtypeObservation` 包装器改变向量化环境中的观测值数据类型。通过指定新的 `dtype`（在本例中为 `float64`），你可以确保观测值的数据类型与模型或算法的要求相匹配，从而提高数据处理的精度和兼容性。

`gymnasium.wrappers.vector.NormalizeObservation` 是一个观测包装器，用于将向量化环境中的观测值标准化，使得每个坐标都以单位方差居中。这在你需要确保观测值符合某些模型预期的输入分布时非常有用，特别是在涉及到神经网络训练的场景中。

### 参数
- `env`: 要包装的向量化环境。
- `epsilon`: 在缩放观测值时使用的稳定性参数，默认值为 `1e-8`。

### 示例代码

以下示例展示了如何使用 `NormalizeObservation` 包装器在 `CartPole-v1` 环境中标准化观测值。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import NormalizeObservation

# 创建一个包含三个子环境的向量化 CartPole-v1 环境
envs = gym.make_vec("CartPole-v1", num_envs=3, vectorization_mode="sync")

# 应用 NormalizeObservation 包装器标准化观测值
envs = NormalizeObservation(envs)

# 重置环境并执行一系列动作
obs, info = envs.reset(seed=123)
_ = envs.action_space.seed(123)
for _ in range(100):
    obs, *_ = envs.step(envs.action_space.sample())

# 打印标准化后的观测值的均值和标准差
print(f"Mean of observations after normalization: {np.mean(obs)}")
print(f"Standard deviation of observations after normalization: {np.std(obs)}")

# 关闭环境
envs.close()
```

### 输出示例

```
Mean of observations after normalization: -0.2359734
Standard deviation of observations after normalization: 1.1938739
```

这个例子展示了如何在向量化环境中使用 `NormalizeObservation` 包装器对观测值进行标准化处理。通过这种方式，观测值的分布被调整为以 0 为中心，具有单位方差，有助于改善学习算法的稳定性和性能。

## 05.Implemented Action wrappers

`gymnasium.wrappers.vector.TransformAction` 是一个行动包装器，用于通过提供给包装器的函数转换向量化环境中的行动。这使得你可以在行动被传递到环境之前对其进行修改或规范化，例如缩放行动的大小或应用某种转换函数，以便更好地适应环境的需求。

### 参数
- `env`: 要包装的向量化环境。
- `func`: 将被应用于所有向量行动的函数。如果此函数转换的行动超出了 `env.action_space` 的范围，则需要提供一个指定向量化环境行动空间的 `action_space`。
- `action_space`: 包装器的行动空间。如果为 None，则假定与 `env.action_space` 相同。

### 示例代码

以下示例展示了如何在 `MountainCarContinuous-v0` 环境中使用 `TransformAction` 包装器来缩小行动值。

```python
import gymnasium as gym
from gymnasium.spaces import Box
from gymnasium.wrappers.vector import TransformAction

# 定义一个函数，用于缩小行动值
def shrink_action(act):
    return act * 0.3

# 创建一个包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 根据转换函数定义新的行动空间
new_action_space = Box(low=shrink_action(envs.action_space.low), high=shrink_action(envs.action_space.high))

# 应用 TransformAction 包装器并传递行动转换函数及新的行动空间
envs = TransformAction(env=envs, func=shrink_action, action_space=new_action_space)

# 重置环境并执行一系列行动
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)
for _ in range(10):
    obs, rew, term, trunc, info = envs.step(envs.action_space.sample())

# 关闭环境
envs.close()

# 打印最终的观测值
print(obs)
```

这个例子通过将原始行动值缩小到原来的 30%，展示了如何使用 `TransformAction` 包装器对行动进行转换。通过这种方式，可以在不直接修改环境代码的情况下调整行动的规模，从而有可能提高学习算法的性能或稳定性。

`gymnasium.wrappers.vector.ClipAction` 是一个向量化环境的行动包装器，用于将连续行动裁剪到有效的 Box 观测空间边界内。这对于防止环境接收到超出其有效操作范围的行动非常有用，可以帮助维持环境的稳定性和避免潜在的错误。

### 参数
- `env`: 要包装的向量化环境。

### 示例代码

以下示例展示了如何使用 `ClipAction` 包装器在 `MountainCarContinuous-v0` 环境中裁剪超出边界的连续行动。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import ClipAction

# 创建一个包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 应用 ClipAction 包装器
envs = ClipAction(envs)

# 重置环境
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)

# 执行一个超出边界的行动数组，并观察裁剪后的行动如何影响环境
obs, rew, term, trunc, info = envs.step(np.array([[5.0], [-5.0], [2.0]]))

# 关闭环境
envs.close()

# 打印观测结果以验证行动是否被正确裁剪
print(obs)
```

在这个例子中，尝试向环境传递三个行动值 `[5.0, -5.0, 2.0]`，这些值超出了 `MountainCarContinuous-v0` 环境中定义的有效行动空间范围。使用 `ClipAction` 包装器后，这些行动将被裁剪到环境的有效行动空间内，从而防止传递无效的行动给环境。这对于自动化训练流程和确保算法稳定性非常有帮助。

`gymnasium.wrappers.vector.RescaleAction` 是一个行动包装器，用于将向量化环境中的连续行动空间线性地重新缩放到指定的 `[min_action, max_action]` 范围内。这使得你可以根据需要调整行动的范围，从而更好地适应不同的学习算法或环境特性。

### 参数
- `env`: 要包装的向量化环境。
- `min_action`: 每个行动的最小值。可以是 numpy 数组或标量。
- `max_action`: 每个行动的最大值。可以是 numpy 数组或标量。

### 示例代码

以下示例展示了如何在 `MountainCarContinuous-v0` 环境中使用 `RescaleAction` 包装器，在不使用行动缩放和使用行动缩放的情况下对环境进行操作。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import RescaleAction

# 创建一个包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 不使用行动缩放
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)
for _ in range(10):
    obs, rew, term, trunc, info = envs.step(0.5 * np.ones((3, 1)))

# 应用 RescaleAction 包装器并设置新的行动范围 [0.0, 1.0]
envs = RescaleAction(envs, 0.0, 1.0)

# 使用行动缩放
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)
for _ in range(10):
    obs, rew, term, trunc, info = envs.step(0.5 * np.ones((3, 1)))

# 关闭环境
envs.close()

# 打印观测结果以验证行动缩放效果
print(obs)
```

在这个例子中，首先进行了一系列操作而没有使用 `RescaleAction` 包装器，然后在应用了 `RescaleAction` 包装器之后再次进行了相同的一系列操作。通过调整行动的范围，可以看到环境的响应（观测结果）如何随着行动缩放的应用而改变。这样的行动缩放可以帮助适配那些期望不同行动范围的算法，提高学习效率和效果。

## 06.Implemented Reward wrappers

`gymnasium.wrappers.vector.TransformReward` 是一个奖励包装器，用于通过自定义函数修改从环境的 `step` 方法返回的奖励。这允许开发者对奖励信号进行转换或缩放，以适应不同的学习算法或目标。

### 参数
- `env`: 要包装的向量化环境。
- `func`: 用于转换奖励的函数。该函数接受原始奖励作为输入，并返回转换后的奖励。

### 示例代码

以下示例展示了如何使用 `TransformReward` 包装器在 `MountainCarContinuous-v0` 环境中对奖励进行线性变换（缩放和位移）。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import TransformReward

# 创建一个包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 定义奖励转换函数
def scale_and_shift(rew):
    # 对奖励进行缩放和位移变换
    return (rew - 1.0) * 2.0

# 应用 TransformReward 包装器并设置奖励转换函数
envs = TransformReward(env=envs, func=scale_and_shift)

# 种子初始化环境和动作空间
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)

# 对环境执行一个步骤，并观察奖励转换效果
obs, rew, term, trunc, info = envs.step(envs.action_space.sample())

# 关闭环境
envs.close()

# 打印观测结果以验证奖励转换效果
print(f"Transformed rewards: {rew}")
```

在这个例子中，我们首先创建了一个包含三个子环境的 `MountainCarContinuous-v0` 环境的向量化版本，然后定义了一个简单的奖励转换函数 `scale_and_shift`，该函数将每个奖励值减去 1.0 后乘以 2.0。通过 `TransformReward` 包装器应用这个函数，我们可以观察到在执行一个步骤之后返回的奖励值是经过转换的。这种方式非常适合于需要调整原始环境奖励信号的场景，以更好地适配特定的学习算法或提高学习效率。

`gymnasium.wrappers.vector.ClipReward` 是一个奖励包装器，用于将向量化环境中的奖励值限制在指定的上下界之间。这对于确保奖励值保持在合理范围内，特别是在奖励值可能非常大或小的环境中，非常有用。这样可以帮助学习算法更稳定地收敛。

### 参数
- `env`: 要包装的向量化环境。
- `min_reward`: 每个步骤的最小奖励值。如果为 `None`，则不应用最小值裁剪。
- `max_reward`: 每个步骤的最大奖励值。如果为 `None`，则不应用最大值裁剪。

### 示例代码

以下示例展示了如何在 `MountainCarContinuous-v0` 环境中使用 `ClipReward` 包装器将奖励值裁剪到 [0.0, 2.0] 的范围内。

```python
import numpy as np
import gymnasium as gym
from gymnasium.wrappers.vector import ClipReward

# 创建一个包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 应用 ClipReward 包装器并设置奖励裁剪范围
envs = ClipReward(envs, 0.0, 2.0)

# 种子初始化环境和动作空间
_ = envs.action_space.seed(123)
obs, info = envs.reset(seed=123)

# 对环境执行十个步骤，并观察裁剪后的奖励值
for _ in range(10):
    obs, rew, term, trunc, info = envs.step(0.5 * np.ones((3, 1)))

# 关闭环境
envs.close()

# 打印裁剪后的奖励值以验证裁剪效果
print(f"Clipped rewards: {rew}")
```

在这个例子中，我们首先创建了一个包含三个子环境的 `MountainCarContinuous-v0` 环境的向量化版本，然后通过 `ClipReward` 包装器应用奖励值裁剪，确保所有返回的奖励值都在 [0.0, 2.0] 范围内。执行十个步骤后，我们可以看到所有返回的奖励值都被成功地裁剪至指定范围。这有助于控制奖励值的波动，使学习过程更加稳定。

`gymnasium.wrappers.vector.NormalizeReward` 是一个奖励包装器，用于标准化向量化环境中的即时奖励，使它们的指数移动平均具有固定的方差。这有助于稳定训练过程，特别是在奖励的尺度不一致或变化很大的环境中。

### 参数
- `env`: 要包装的向量化环境。
- `epsilon`: 在奖励标准化时用作稳定参数的一个小正数，默认为 `1e-8`。
- `gamma`: 用于计算指数移动平均的折扣因子，默认为 `0.99`。

### 示例代码

以下示例演示了如何在 `MountainCarContinuous-v0` 环境中应用 `NormalizeReward` 包装器来标准化奖励值。

```python
import gymnasium as gym
import numpy as np

# 创建包含三个子环境的向量化 MountainCarContinuous-v0 环境
envs = gym.make_vec("MountainCarContinuous-v0", num_envs=3)

# 应用 NormalizeReward 包装器
envs = gym.wrappers.vector.NormalizeReward(envs)

# 重置环境
_ = envs.reset(seed=123)
_ = envs.action_space.seed(123)

# 执行多次步骤并收集奖励
episode_rewards = []
for _ in range(100):
    _, reward, _, _, _ = envs.step(envs.action_space.sample())
    episode_rewards.append(reward)

# 关闭环境
envs.close()

# 计算和显示标准化后的奖励的平均值和标准差
print(f"Normalized reward mean: {np.mean(episode_rewards)}")
print(f"Normalized reward std: {np.std(episode_rewards)}")
```

### 说明
这个示例中，我们首先创建了一个包含三个子环境的 `MountainCarContinuous-v0` 环境的向量化版本，然后通过 `NormalizeReward` 包装器应用奖励标准化，确保所有返回的奖励值的指数移动平均具有固定的方差。执行一系列步骤后，我们计算了标准化后的奖励值的平均值和标准差，以验证标准化的效果。这种方法可以帮助学习算法更稳定地收敛，尤其是在奖励值波动很大的环境中。

## 07.Implemented Data Conversion wrappers

`gymnasium.wrappers.vector.JaxToNumpy` 是一个向量化环境包装器，用于将基于 JAX 的向量化环境包装起来，使其可以通过 NumPy 数组进行交互。这允许使用基于 NumPy 的工具和库来处理来自环境的数据，同时享受 JAX 环境的高性能计算优势。

### 参数
- `env`: 要包装的基于 JAX 的向量化环境。

### 示例代码

以下示例展示了如何创建一个基于 JAX 的向量化环境，并将其包装为可以通过 NumPy 数组进行交互的环境。

```python
import gymnasium as gym

# 假设 "JaxEnv-vx" 是一个基于 JAX 的环境，这里用作示例
# 请替换为实际的基于 JAX 的向量化环境名称
envs = gym.make_vec("JaxEnv-vx", num_envs=3)  # 创建包含三个子环境的向量化环境
envs = gym.wrappers.vector.JaxToNumpy(envs)   # 应用 JaxToNumpy 包装器

# 重置环境并进行一些步骤
obs, info = envs.reset(seed=123)
for _ in range(5):
    actions = envs.action_space.sample()  # 生成 NumPy 数组形式的动作
    obs, rewards, terminations, truncations, infos = envs.step(actions)
    print(f"Observations: {obs}, Rewards: {rewards}")

# 关闭环境
envs.close()
```

### 功能
- **动作输入**：通过 NumPy 数组提供动作。
- **观测、奖励、终止和截断信号返回**：以 NumPy 数组的形式返回观测值、奖励、是否终止和是否截断的信号。

这个包装器特别适用于那些希望将 JAX 环境与仅支持 NumPy 数组输入和输出的库或算法结合使用的开发者。

`gymnasium.wrappers.vector.JaxToTorch` 是一个向量化环境包装器，用于将基于 JAX 的向量化环境包装起来，使其可以通过 PyTorch 张量进行交互。这允许将 JAX 环境的数据直接用于基于 PyTorch 的机器学习模型和算法中，而不需要手动在 JAX 数组和 PyTorch 张量之间进行转换。

### 参数
- **`env`**: 要包装的基于 JAX 的向量化环境。
- **`device`**: 指定 PyTorch 张量应该移动到的设备。如果没有提供，张量将保留在它们原始的设备上。

### 示例代码

以下示例展示了如何创建一个基于 JAX 的向量化环境，并将其包装为可以通过 PyTorch 张量进行交互的环境，同时指定使用 PyTorch 设备。

```python
import gymnasium as gym
import torch

# 假设 "JaxEnv-vx" 是一个基于 JAX 的环境，这里用作示例
# 请替换为实际的基于 JAX 的向量化环境名称
envs = gym.make_vec("JaxEnv-vx", num_envs=3)  # 创建包含三个子环境的向量化环境
envs = gym.wrappers.vector.JaxToTorch(envs, device=torch.device('cuda'))   # 应用 JaxToTorch 包装器并指定设备

# 重置环境并进行一些步骤
obs, info = envs.reset(seed=123)
for _ in range(5):
    actions = torch.tensor(envs.action_space.sample(), device=torch.device('cuda'))  # 生成 PyTorch 张量形式的动作
    obs, rewards, terminations, truncations, infos = envs.step(actions)
    print(f"Observations: {obs}, Rewards: {rewards}")

# 关闭环境
envs.close()
```

### 功能
- **动作输入**: 通过 PyTorch 张量提供动作。
- **观测、奖励、终止和截断信号返回**: 以 PyTorch 张量的形式返回观测值、奖励、是否终止和是否截断的信号。

这个包装器特别适用于那些希望将 JAX 环境与基于 PyTorch 的机器学习模型和算法结合使用的开发者。通过指定设备，开发者可以确保所有的数据和计算都在指定的 GPU 或 CPU 上执行，以优化性能和资源利用率。

`gymnasium.wrappers.vector.NumpyToTorch` 是一个包装器，它允许将基于 NumPy 的向量化环境转换为可以通过 PyTorch 张量进行交互的环境。这使得基于 NumPy 的环境可以无缝地集成到基于 PyTorch 的机器学习和深度学习模型中，从而简化了数据处理和模型训练的流程。

### 参数
- **`env`**: 要包装的基于 NumPy 的向量化环境。
- **`device`**: 指定 PyTorch 张量应该移动到的设备。这可以是 `"cpu"` 或 `"cuda"` 等。如果没有提供，张量将保留在它们原始的设备上。

### 示例代码

以下示例展示了如何将基于 NumPy 的向量化 `CartPole-v1` 环境转换为可以通过 PyTorch 张量进行交互的环境，并指定使用 PyTorch 设备。

```python
import torch
import gymnasium as gym
from gymnasium.wrappers.vector import NumpyToTorch

# 创建一个包含三个子环境的向量化 CartPole-v1 环境
envs = gym.make_vec("CartPole-v1", num_envs=3)

# 应用 NumpyToTorch 包装器并指定设备
envs = NumpyToTorch(envs, device=torch.device('cuda'))

# 重置环境并获取 PyTorch 张量形式的初始观测值
obs, _ = envs.reset(seed=123)
print(f"Observation type after reset: {type(obs)}")

# 生成 PyTorch 张量形式的动作，并执行一步动作
action = torch.tensor(envs.action_space.sample(), device=torch.device('cuda'))
obs, reward, terminated, truncated, info = envs.step(action)

# 关闭环境
envs.close()

# 打印数据类型以验证转换
print(f"Observation type after step: {type(obs)}")
print(f"Reward type: {type(reward)}")
print(f"Terminated type: {type(terminated)}")
print(f"Truncated type: {type(truncated)}")
```

### 功能
- **动作输入**: 通过 PyTorch 张量提供动作。
- **观测、奖励、终止和截断信号返回**: 以 PyTorch 张量的形式返回观测值、奖励、是否终止和是否截断的信号。

这个包装器非常适用于希望利用 PyTorch 的强大功能，如自动梯度计算和 GPU 加速，来训练和评估他们的模型的开发者。通过指定设备，可以确保所有的数据和计算都在指定的 GPU 或 CPU 上执行，优化了性能和资源利用率。